In [1]:
import pandas as pd
import smtplib
import os
import json
import matplotlib.pyplot as plt
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication
from email import encoders
from email.utils import make_msgid

In [2]:
# Configuración de comportamiento
ORIGEN = 'misiontic.noresponder@outlook.com'
ASUNTO = 'ESTADISTICAS DE RENDIMIENTO DE GRUPOS'
SERVIDOR_SMTP = 'smtp-mail.outlook.com:587'
CORREO_TEST = True
RUTA = './INPUT/INFO_CURSOS/LISTA_CURSOS_FORMADORES_ESTADISTICAS.xlsx' if not CORREO_TEST else './INPUT/INFO_CURSOS/TEST_2.xlsx'
COLUMNA_CORREO = 'CORREO_DOCENTE'
RETO_ACTUAL_IDEAL = 3
COLUMNAS = [
    'CURSO',
    'TOTAL_ESTUDIANTES',
    'ASISTENCIA_PROMEDIO',
    'N_EST_RETO_0',
    'N_EST_RETO_%_0',
    'N_EST_RETO_1',
    'N_EST_RETO_%_1',
    'N_EST_RETO_2',
    'N_EST_RETO_%_2',
    'N_EST_RETO_3',
    'N_EST_RETO_%_3',
    'N_EST_RETO_4',
    'N_EST_RETO_%_4',
    'N_EST_RETO_5',
    'N_EST_RETO_%_5'
]

In [3]:
MSG_HEAD = """<html><body><p>Bucaramanga, 19 de mayo de 2022<br>
Respetado profesional<br>
<b>{formador}</b><br>
Programa Misión TIC 2022</p>

"""
MSG_BODY = """<p>En atención al asunto, adjunto a este correo va un informe con el rendimiento de su(s) grupo(s). A continuación se presenta un detalle de cada columna:</p>
<ul>
<li><b>CURSO:</b> El código del curso.</li>
<li><b>TOTAL_ESTUDIANTES:</b> El número de estudiantes matriculados en el curso, a fecha de corte del 9 de mayo.</li>
<li><b>ASISTENCIA_PROMEDIO:</b> El índice de estudiantes que asisten por curso. Va de cero (0) a uno (1), siendo cero una INASISTENCIA total y uno la asistencia de todos los tripulantes.</li>
<li><b>N_EST_RETO_0:</b> El número de estudiantes que NO han presentado ningún reto.</li>
<li><b>N_EST_RETO_%_0:</b> El índice de estudiantes que NO han presentado ningún reto. Va de cero (0) a uno (1), siendo cero que todos los tripulantes presentaron, al menos, un reto, y uno que ningún tripulante ha presentado ningún reto.</li>
<li><b>N_EST_RETO_1:</b> El número de estudiantes que han presentado, AL MENOS, un (1) reto.</li>
<li><b>N_EST_RETO_%_1:</b> El índice de estudiantes que han presentado, AL MENOS, un (1) reto. Va de cero (0) a uno (1), siendo cero que NINGÚN estudiante ha hecho, al menos, un reto, y uno que todos los estudiantes han realizado un reto.</li>
</ul>
<p>Las demás columnas aplican el mismo principio.</p>
"""

MSG_FOOTER = """<p>ESTE CORREO SE ENVIÓ DE FORMA AUTOMÁTICA, NO RESPONDA A ESTA DIRECCIÓN O CORREO. Cualquier información, contacte a misiontic.monitor@uis.edu.co o rectoria.misiontic@uis.edu.co.</p>
<p>Tecnología diseñada por <a href="https://nuwebs.com.co">Nuwebs</a></p>
<p>Bucaramanga, Colombia. +57 3184301032</p>
</body></html>"""

In [4]:
f = open('CREDENCIALES.json')
CREDENCIALES = json.load(f)
f.close()
datos = pd.read_excel(RUTA, engine = 'openpyxl')

In [5]:
datos

,CURSO,NOMBRE_DOCENTE,CORREO_DOCENTE,TOTAL_ESTUDIANTES,ASISTENCIA_PROMEDIO,N_EST_RETO_0,N_EST_RETO_%_0,N_EST_RETO_1,N_EST_RETO_%_1,N_EST_RETO_2,N_EST_RETO_%_2,N_EST_RETO_3,N_EST_RETO_%_3,N_EST_RETO_4,N_EST_RETO_%_4,N_EST_RETO_5,N_EST_RETO_%_5
0,A22,ROY HERNANDO LLAMAS MUNOZ,ra.douglas@hotmail.com,46,0.269565,29,0.630435,7,0.152174,7,0.152174,3,0.065217,0,0,0,0
1,A26,DAVID EDUARDO FLOREZ GOMEZ,ra.douglas@hotmail.com,68,0.588235,39,0.573529,17,0.250000,12,0.176471,0,0.000000,0,0,0,0
2,A34,ROGERIO ORLANDO BELTRAN CASTRO,ra.douglas123@gmail.com,67,0.453731,36,0.537313,6,0.089552,24,0.358209,1,0.014925,0,0,0,0


In [6]:
def generarCorreo(origen, destino, asunto):
    multipart = MIMEMultipart('related') # Posible quitar related
    multipart['From'] = origen
    multipart['To'] = destino
    multipart['Subject'] = asunto
    multipart['X-Priority'] = '2'
    multipart.preamble = '====================================================='
    
    return multipart

def getCuerpoCorreo(multipart, formador, df, imgs = []): 
    msgAlternative = MIMEMultipart('alternative') #Posible quitar esto
    multipart.attach(msgAlternative)
    
    cuerpoAdicional = ''
    cont = 0
    contImgs = len(imgs)
    
    multipartImgs = []
    for index, row in df.iterrows():
        grupo = row['CURSO']
        asistencia = row['ASISTENCIA_PROMEDIO'] * 100
        sinRetosN = row['N_EST_RETO_0']
        sinRetosP = row['N_EST_RETO_%_0'] * 100
        retoActualN = 0
        retoActualP = 0
        for i in range (RETO_ACTUAL_IDEAL, 5 + 1):
            retoActualN += row['N_EST_RETO_' + str(i)]
            retoActualP += row['N_EST_RETO_%_' + str(i)]
        retoActualP *= 100
        cuerpoAdicional += f"<h4><b>GRUPO {grupo}</b></h4>"
        if cont < contImgs:
            imageId = make_msgid()
            cuerpoAdicional += '<img src="cid:{image_cid}">'.format(image_cid=imageId[1:-1])
            with open(imgs[cont], 'rb') as f:
                msgImage = MIMEImage(f.read())
            msgImage.add_header('Content-ID', imageId)
            multipartImgs.append(msgImage)
        cont += 1
        cuerpoAdicional += '<ul>\n'
        if sinRetosP > 0.3*100:
            cuerpoAdicional += """<li>El grupo {grupo} tiene un <b>ALTO</b> porcentaje ({porcentaje:.2f}%) de tripulantes ({numero}) que aún <b>NO</b> han presentado algún reto.</li>\n""".format(grupo = grupo, numero = sinRetosN, porcentaje = sinRetosP)
        if retoActualP < 0.4*100:
            cuerpoAdicional += """<li>El grupo {grupo} tiene un <b>BAJO</b> porcentaje ({porcentaje:.2f}%) de tripulantes ({numero}) que están al día con el número de retos mínimo ideal ({retoActual}) para esta semana.</li>\n""".format(grupo = grupo, numero = retoActualN, porcentaje = retoActualP, retoActual = str(RETO_ACTUAL_IDEAL))
        if asistencia < 0.5*100:
            cuerpoAdicional += """<li>El grupo {grupo} tiene un <b>BAJO</b> porcentaje ({porcentaje:.2f}%) de asistencia.</li>\n""".format(grupo = grupo, porcentaje = asistencia)
        cuerpoAdicional += '</ul>\n'
    #if cuerpoAdicional != '':
    #    cuerpoAdicional = '\nA continuación, se presentan algunas observaciones relevantes, por favor, elabore un plan de acción que las atienda:</p><br>' + cuerpoAdicional
    cuerpo = MSG_HEAD.format(formador = formador) + MSG_BODY + cuerpoAdicional + MSG_FOOTER
    cuerpo = MIMEText(cuerpo, 'html', 'utf-8')
    msgAlternative.attach(cuerpo)
    return multipart, cuerpo, multipartImgs

def getAdjuntos(multipart, imgs = [], adjuntos = []):
    for img in imgs:
        multipart.attach(img)
    for adjunto in adjuntos:
        f = open(adjunto, 'rb')
        nFile = MIMEApplication(f.read(), 'vnd.ms-excel')
        f.close()
        encoders.encode_base64(nFile)
        nFile.add_header('Content-Disposition', 'attachment', filename=os.path.basename(adjunto))
        multipart.attach(nFile)
    return multipart

def graficar(df):
    rutasImagenes = []
    for index, row in df.iterrows():
        nombreCurso = row['CURSO']
        retos = []
        for i in range(6):
            retos.append(row['N_EST_RETO_' + str(i)])
        fig, ax = plt.subplots(figsize=(10,5), dpi=100);
        ax.set_title(f"Retos resueltos grupo {nombreCurso}")
        ax.set_ylabel('Número de estudiantes')
        ax.grid(True, zorder=0)
        barras = ax.bar(['Sin retos', 'Un reto', 'Dos retos', 'Tres retos', 'Cuatro retos', 'Cinco retos'], retos, zorder=3);
        i = 0
        for barra in barras:
            barra.set_color('gray' if i < RETO_ACTUAL_IDEAL else 'cyan')
            i+=1
            alto = barra.get_height()
            ancho = barra.get_width()
            ajuste = 0.07 if alto < 10 else 0.12
            x = barra.get_x() + (ancho*(1-ajuste))/2
            y = alto + 0.4
            ax.text(x,y, alto)
        barras[0].set_color('red')
        barras[RETO_ACTUAL_IDEAL].set_color('green')
        ruta = './Temp/Hist-Retos-' + nombreCurso + '.png'
        rutasImagenes.append(ruta)
        fig.savefig(ruta)
    return rutasImagenes

In [7]:
%matplotlib agg
destinos = datos[COLUMNA_CORREO].unique()
servidor = smtplib.SMTP(SERVIDOR_SMTP)
servidor.starttls()
servidor.login(CREDENCIALES['USUARIO'], CREDENCIALES['PASS'])
cont = 1
for destino in destinos:
    dfD = datos[datos[COLUMNA_CORREO] == destino]
    nombreFormador = dfD['NOMBRE_DOCENTE'].iloc[0]
    adjuntos = []
    nombreArchivo = 'ESTADISTICAS_GRUPOS_' + nombreFormador.replace(' ', '_')
    ruta = './Temp/' + nombreArchivo + '.xlsx'
    dfD[COLUMNAS].to_excel(ruta, index = False)
    adjuntos.append(ruta)
    imgs = graficar(dfD)
    
    base = generarCorreo(ORIGEN, destino, ASUNTO)
    base, contenido, mImgs = getCuerpoCorreo(base, nombreFormador, dfD, imgs)
    base = getAdjuntos(base, mImgs, adjuntos)
    
    servidor.sendmail(ORIGEN, destino, base.as_string())
    print (cont, 'Enviado a', nombreFormador, destino)
    cont += 1
servidor.quit()

1 Enviado a ROY HERNANDO LLAMAS MUNOZ ra.douglas@hotmail.com
2 Enviado a ROGERIO ORLANDO BELTRAN CASTRO ra.douglas123@gmail.com


(221, b'2.0.0 Service closing transmission channel')